<a href="https://colab.research.google.com/github/HoYoungChun/financial_data_analysis/blob/main/NCAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NCAV(Net Current Asset Value)

### Basic Setting

In [1]:
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import urllib.request
# !pip install bs4
# !pip install urllib


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pandas as pd

csv_url = '/content/drive/MyDrive/Colab/financial_data_analysis/company.csv'

company = pd.read_csv(csv_url, encoding='utf-8')

In [4]:
SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
print(SNAP_URL)

https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701


In [5]:
FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'
print(FS_URL)

https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701


In [6]:
sample_code = '035720'

In [11]:
fs = urllib.request.urlopen(FS_URL.format(sample_code)).read()
fs_soup = BeautifulSoup(fs,'html.parser')

In [13]:
snap = urllib.request.urlopen(SNAP_URL.format(sample_code)).read()
snap_soup = BeautifulSoup(snap,'html.parser')

### Market Value

In [24]:
snap_soup.find('div',{'id':"svdMainGrid1"}).find_all('td',{'class':'r'})

[<td class="r">157,500/ <span class="tcr">+500</span></td>,
 <td class="cle r"> 2,045,164</td>,
 <td class="r">169,500/ 62,222</td>,
 <td class="cle r"> 3,225</td>,
 <td class="r"><span class="tcr">+27.02</span>/ <span class="tcr">+44.24</span>/ <span class="tcr">+98.40</span>/ <span class="tcr">+161.13</span>
 <input id="c3M" type="hidden" value="44.24"/><input id="c1Y" type="hidden" value="161.13"/><input id="c3Y" type="hidden" value="585.32"/></td>,
 <td class="cle r">32.03</td>,
 <td class="r">699,189</td>,
 <td class="cle r">0.74595</td>,
 <td class="r">699,189</td>,
 <td class="cle r">100</td>,
 <td class="r">444,460,230/ 0</td>,
 <td class="cle r">323,743,012 / 72.93</td>]

In [23]:
!cd drive
!ls

drive  sample_data
